<a href="https://colab.research.google.com/github/Nutanpatil06/Fine-Tuning-LLM-with-LLaMA-Factory/blob/main/Fine_Tuning_LLM_with_LLaMA_Factory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/hiyouga/LLaMA-Factory.git

In [ ]:
!pwd

In [ ]:
%cd /content/LLaMA-Factory/

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
!pip install -e .

In [ ]:
!pip install bitsandbytes>=0.39.0

In [ ]:
!ls /content/LLaMA-Factory/src/

In [ ]:
import os
os.environ["GRADIO_SHARE"] = "1"

In [ ]:
!huggingface-cli login

In [ ]:
# !pip install accelerate transformers peft

In [ ]:
!python /content/LLaMA-Factory/src/webui.py

In [ ]:
# from llamafactory.webui.interface import create_ui

In [ ]:
# ui = create_ui()

In [ ]:
ui.launch(share=True)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

#Base model (same as you used in training)
base_model = "google/gemma-1.1-2b-it"

#Path to your trained LoRA adapter
lora_path = "/content/LLaMA-Factory/saves/Gemma-1.1-2B-Instruct/lora/train_2026-01-06-09-51-36"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map='auto',
)

# Apply LoRA on top of base model
model = PeftModel.from_pretrained(
    model,
    lora_path
)

In [ ]:
# Load tokenizer

# Enable evaluation mode
model.eval()

# --- Inference prompt ---
prompt = "Can you tell what ls -l would display?"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate output
outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=0.7,
    do_sample=True,
)

# Decode output
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# CLI Based Methods

In [ ]:
# !python -m llamafactory.cli train \
#   --model_name_or_path google/gemma-1.1-2b-it \
#   --template gemma \
#   --stage sft \
#   --finetuning_type lora \
#   --dataset yahma/alpaca-cleaned \
#   --output_dir output/my-gemma-qlora \
#   --cutoff_len 2048 \
#   --per_device_train_batch_size 1 \
#   --gradient_accumulation_steps 8 \
#   --num_train_epochs 1 \
#   --learning_rate 5e-5 \
#   --lora_rank 64 \
#   --lora_alpha 16 \
#   --lora_dropout 0.05 \
#   --quantization_bit 4 \
#   --fp16 True \
#   --gradient_checkpointing True \
#   --save_strategy epoch \
#   --save_steps 0 \
#   --save_total_limit 3 \
#   --logging_steps 10

In [ ]:
%cd /content/LLaMA-Factory

In [ ]:
!pwd

In [ ]:
!nvidia-smi

In [ ]:
%env CUDA_LAUNCH_BLOCKING=1

In [ ]:
!python -m llamafactory.cli train train_gemma_qlora.yaml

In [ ]:
!ls output/my-gemma-qlora

In [ ]:
#!python -m llamafactory.cli train test.yaml

In [ ]:
# !python -m llamafactory.cli train \
#   --model_name_or_path google/gemma-1.1-2b-it \
#   --template gemma \
#   --finetuning_type lora \
#   --dataset yahma/alpaca-cleaned \
#   --output_dir output/debug \
#   --per_device_train_batch_size 1 \
#   --gradient_accumulation_steps 8 \
#   --num_train_epochs 1 \
#   --lora_rank 32 --lora_alpha 16 --lora_dropout 0.05 \
#   --save_strategy epoch --save_total_limit 1

In [ ]:
# !df -h
# !touch output/debug/testfile && ls output/debug

In [ ]:
!ls output/test-lora

In [ ]:
import os
print(os.listdir("output/test-lora"))

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

base = "google/gemma-1.1-2b-it"
adapter = "/content/LLaMA-Factory/gemma_lora_sft_output"   # your LoRA folder path

tokenizer = AutoTokenizer.from_pretrained(base)
model = AutoModelForCausalLM.from_pretrained(base, device_map="auto", torch_dtype=torch.float16)

model = PeftModel.from_pretrained(model, adapter)

prompt = "Explain what is QLoRA"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=200)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!python -m llamafactory.cli export train_gemma_qlora.yaml